# Práctica 1: Aprendizaje Supervisado
## Dataset: Estimation of Obesity Levels Based on Eating Habits and Physical Condition

**Autor:** Jordi Blasco Lozano  
**DNI:** 74527208D  
**Email:** jbl42@alu.ua.es  
**Asignatura:** Aprendizaje Avanzado - Curso 2025/2026  
**Universidad de Alicante - Escuela Politécnica Superior**

> Nota: se reutiliza el mismo dataset y el mismo preprocesado de la Práctica 0.  
> El objetivo es comparar modelos supervisados (SVM, árboles y ensembles) y analizar el efecto de hiperparámetros.


## 1. Carga de datos y configuración

En esta primera sección importamos todas las librerías necesarias, definimos funciones auxiliares para guardar tablas y figuras, y cargamos el dataset. Reutilizamos el mismo CSV de la Práctica 0 (*Estimation of Obesity Levels Based on Eating Habits and Physical Condition*) para garantizar comparabilidad directa con los resultados anteriores.

In [1]:
# Imports y configuración
from pathlib import Path
from time import perf_counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay
)

# Modelos
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, AdaBoostClassifier
)
from sklearn.inspection import permutation_importance

# Reproducibilidad
RANDOM_STATE = 42

# Carpetas de salida
BASE_DIR = Path('.').resolve()
IMAGES_DIR = BASE_DIR / 'images'
TABLES_DIR = BASE_DIR / 'tables'
IMAGES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)


In [2]:
def find_dataset_path() -> Path:
    """Busca el CSV en las rutas típicas del repo de prácticas."""
    candidates = [
        BASE_DIR / 'data' / 'ObesityDataSet_raw_and_data_sinthetic.csv',
        BASE_DIR.parent / 'practica_0' / 'data' / 'ObesityDataSet_raw_and_data_sinthetic.csv',
        BASE_DIR.parent / 'practica0' / 'data' / 'ObesityDataSet_raw_and_data_sinthetic.csv',
        BASE_DIR / 'ObesityDataSet_raw_and_data_sinthetic.csv',
    ]
    for p in candidates:
        if p.exists():
            return p
    raise FileNotFoundError(
        "No se encontró el dataset.\n\n"
        "Rutas probadas:\n - " + "\n - ".join(str(p) for p in candidates) + "\n\n"
        "Asegúrate de tener el CSV en practica_0/data o en practica_1/data."
    )

def save_table(df: pd.DataFrame, name: str) -> Path:
    """Guarda una tabla como LaTeX (solo tabular) en ./tables/{name}.tex"""
    out = TABLES_DIR / f"{name}.tex"
    latex = df.to_latex(index=False, escape=True, na_rep='-', float_format="%.4f", bold_rows=False)
    out.write_text(latex, encoding='utf-8')
    return out

def save_fig(path: Path, dpi: int = 300):
    plt.tight_layout()
    plt.savefig(path, dpi=dpi, bbox_inches='tight')
    plt.close()

def metrics_row(y_true, y_pred) -> dict:
    return {
        'Test Accuracy': accuracy_score(y_true, y_pred),
        'Precision (w)': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'Recall (w)': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'F1 (w)': f1_score(y_true, y_pred, average='weighted', zero_division=0),
    }

def plot_confusion(y_true, y_pred, class_names, filename: str, title: str):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names, ax=ax)
    ax.set_xlabel('Predicho')
    ax.set_ylabel('Real')
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=45)
    ax.tick_params(axis='y', rotation=0)
    save_fig(IMAGES_DIR / filename)



In [3]:
DATA_PATH = find_dataset_path()
df = pd.read_csv(DATA_PATH)
print('Ruta:', DATA_PATH)
print('Shape original:', df.shape)
df.head()


Ruta: C:\Users\jordi\Documents\UNI\IA\3erAnyo\2doCuatri\aprendizaje_avanzado\practicas\practica_0\data\ObesityDataSet_raw_and_data_sinthetic.csv
Shape original: (2111, 17)


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


## 2. Preprocesado (idéntico a Práctica 0)

Tal como indica el enunciado, el preprocesado se da por supuesto y no es necesario detallarlo de nuevo. Para garantizar comparabilidad con la Práctica 0, se aplica exactamente el mismo pipeline de limpieza:

1. **Eliminación de outliers en Weight y Height** mediante el método IQR (Rango Intercuartílico).
2. **Transformación Box-Cox en Age** y posterior filtrado IQR sobre la distribución transformada.
3. **Encoding de variables categóricas:** *Label Encoding* para las binarias (Gender, family_history, FAVC, SMOKE, SCC) y *One-Hot Encoding* con `drop_first=True` para las multiclase (CAEC, CALC, MTRANS).
4. **Split train/test estratificado** 80/20 con `random_state=42`.
5. **Filtrado de outliers en train** con Z-score (|z| > 3) sobre las variables numéricas originales.

> **Nota sobre el escalado:** la estandarización no se aplica manualmente aquí, sino dentro de `Pipeline` (StandardScaler + modelo) para SVM y modelos lineales. Esto elimina el riesgo de *Data Leakage* y garantiza que en cada fold de validación cruzada la estandarización se recalcula de forma independiente.

In [4]:
target_col = 'NObeyesdad'

# 1) IQR en Weight y Height
df_clean = df.copy()
mask_clean = pd.Series(True, index=df_clean.index)
for col in ['Weight', 'Height']:
    q1 = df_clean[col].quantile(0.25)
    q3 = df_clean[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    mask_clean &= (df_clean[col] >= lower) & (df_clean[col] <= upper)

df_clean = df_clean.loc[mask_clean].reset_index(drop=True)

# 2) Box-Cox en Age + IQR post
df_clean['Age'], lambda_age = stats.boxcox(df_clean['Age'] + 1)
q1 = df_clean['Age'].quantile(0.25)
q3 = df_clean['Age'].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
df_clean = df_clean[(df_clean['Age'] >= lower) & (df_clean['Age'] <= upper)].reset_index(drop=True)

# 3) Separar X/y + encoding del target
X = df_clean.drop(columns=[target_col]).copy()
y = df_clean[target_col].copy()

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)
class_names = le_target.classes_.tolist()

# 4) Encoding de features (idéntico a práctica 0)
X_processed = X.copy()

binary_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
for col in binary_cols:
    if col in X_processed.columns:
        le = LabelEncoder()
        X_processed[col] = le.fit_transform(X_processed[col])

multi_cat_cols = ['CAEC', 'CALC', 'MTRANS']
X_processed = pd.get_dummies(X_processed, columns=multi_cat_cols, drop_first=True)

# Columnas numéricas originales (para Z-score post-split)
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != target_col]

# 5) Split 80/20 estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_encoded,
    test_size=0.2, random_state=RANDOM_STATE, stratify=y_encoded
)

# 6) Filtrado de outliers en train con Z-score en columnas numéricas originales
scaler_temp = StandardScaler()
X_train_z = scaler_temp.fit_transform(X_train[numeric_cols])
mask_inliers = (np.abs(X_train_z) <= 3).all(axis=1)

rows_before = len(X_train)
X_train = X_train.loc[mask_inliers].reset_index(drop=True)
y_train = y_train[mask_inliers]
rows_removed = rows_before - len(X_train)

print('Shape tras limpieza:', df_clean.shape)
print('Shape tras encoding:', X_processed.shape)
print(f'Train: {X_train.shape} | Test: {X_test.shape}')
print(f'Outliers eliminados en train (Z>3): {rows_removed}')


Shape tras limpieza: (2107, 17)
Shape tras encoding: (2107, 23)
Train: (1685, 23) | Test: (422, 23)
Outliers eliminados en train (Z>3): 0


## 3. Parte 1 — Support Vector Machines (SVM)

Las SVM buscan el hiperplano que maximiza el margen entre clases. En problemas no linealmente separables, utilizan funciones *kernel* para proyectar los datos a un espacio de mayor dimensionalidad donde sí son separables. Los principales hiperparámetros son:

- **C:** controla el trade-off entre maximizar el margen y minimizar errores de clasificación. Un C alto genera un margen estrecho con pocos errores (riesgo de *overfitting*), mientras que un C bajo genera un margen amplio pero permite más errores (*underfitting*).
- **gamma (γ):** en kernels RBF y polinomial, controla el alcance de influencia de cada muestra. Un gamma alto implica influencia local (fronteras complejas), y un gamma bajo implica influencia amplia (fronteras suaves).
- **degree:** grado del polinomio en el kernel polinomial.

> **Importante:** SVM es sensible a la escala de las variables, por lo que siempre encapsulamos un `StandardScaler` dentro del `Pipeline` antes del modelo.

### 3.1 SVM con diferentes kernels (parámetros por defecto)

Empezamos entrenando modelos SVM con los cuatro kernels principales (Linear, Polinomial grado 3, RBF y Sigmoid) usando los hiperparámetros por defecto de scikit-learn. Para cada kernel medimos: accuracy en validación cruzada 5-Fold, accuracy y métricas ponderadas en test, número de vectores de soporte y tiempo de entrenamiento.

In [5]:
kernels = {
    'Linear':     SVC(kernel='linear', random_state=RANDOM_STATE),
    'Poly (d=3)': SVC(kernel='poly', degree=3, random_state=RANDOM_STATE),
    'RBF':        SVC(kernel='rbf', random_state=RANDOM_STATE),
    'Sigmoid':    SVC(kernel='sigmoid', random_state=RANDOM_STATE),
}

rows = []
for name, svc in kernels.items():
    pipe = Pipeline([('scaler', StandardScaler()), ('svm', svc)])

    # CV
    cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean, cv_std = cv_scores.mean(), cv_scores.std()

    # Fit + test
    t0 = perf_counter()
    pipe.fit(X_train, y_train)
    fit_time = perf_counter() - t0

    y_pred = pipe.predict(X_test)
    m = metrics_row(y_test, y_pred)

    n_sv = int(pipe.named_steps['svm'].n_support_.sum())
    rows.append({
        'Kernel': name,
        'CV Accuracy': cv_mean,
        'CV Std': cv_std,
        **m,
        'Nº SV': n_sv,
        '% Train SV': 100 * n_sv / len(X_train),
        'Fit Time (s)': fit_time,
    })

df_svm_kernels = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
df_svm_kernels


,Kernel,CV Accuracy,CV Std,Test Accuracy,Precision (w),Recall (w),F1 (w),Nº SV,% Train SV,Fit Time (s)
0,Linear,0.941840,0.024940,0.938389,0.939125,0.938389,0.938064,565,33.531157,0.048107
2,RBF,0.852819,0.023183,0.872038,0.880678,0.872038,0.874658,1089,64.629080,0.050225
1,Poly (d=3),0.745401,0.020507,0.744076,0.761190,0.744076,0.738924,1202,71.335312,0.053569
3,Sigmoid,0.640950,0.032014,0.651659,0.649538,0.651659,0.650088,1217,72.225519,0.049305


El kernel **Linear** obtiene el mejor resultado con parámetros por defecto (CV = 0.9418, Test = 0.9384), superando claramente al resto. Esto puede deberse a que el preprocesado (encoding + estandarización) ya genera un espacio de 23 dimensiones donde las clases son razonablemente separables por fronteras lineales, tal como sugerían los resultados de Regresión Logística en la Práctica 0 (F1 = 0.9359 con C = 10).

El kernel **RBF**, que suele ser el más versátil, queda en segundo lugar (CV = 0.8528) con los parámetros por defecto. Sin embargo, esto no significa que sea peor: sus hiperparámetros C y gamma necesitan ajustarse, cosa que haremos en la siguiente sección.

El kernel **Sigmoid** es el peor con diferencia (CV = 0.6410). Este kernel se comporta como un perceptrón multicapa de una capa oculta y suele dar problemas de rendimiento cuando no se ajustan cuidadosamente sus parámetros.

También es interesante observar el número de **vectores de soporte (SV)**: el kernel lineal usa 565 SV (33.5% del training set), mientras que los kernels no lineales necesitan muchos más (64–72%). A mayor número de SV, mayor complejidad efectiva del clasificador y mayor coste de inferencia.

In [6]:
save_table(df_svm_kernels[['Kernel','CV Accuracy','Test Accuracy','Precision (w)','Recall (w)','F1 (w)','Nº SV','% Train SV']], 'svm_kernels')


WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/svm_kernels.tex')

### 3.2 Optimización del kernel RBF (GridSearchCV)

El kernel RBF (`exp(-γ·||x-x'||²)`) suele ser el más efectivo para datos no lineales, pero depende fuertemente de dos hiperparámetros:

- **C** (regularización): valores altos → margen pequeño, se ajusta más a los datos.
- **gamma (γ)**: valores altos → cada muestra influye solo en su vecindad cercana, produciendo fronteras más complejas.

Utilizamos `GridSearchCV` con CV 5-Fold para explorar combinaciones de C ∈ {0.1, 1, 10, 100} y γ ∈ {0.001, 0.01, 0.1, 1, 'scale', 'auto'}.

In [7]:
pipe_rbf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', random_state=RANDOM_STATE))
])

param_grid_rbf = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto']
}

grid_rbf = GridSearchCV(
    pipe_rbf, param_grid_rbf,
    cv=5, scoring='accuracy', n_jobs=-1, refit=True
)

t0 = perf_counter()
grid_rbf.fit(X_train, y_train)
grid_time = perf_counter() - t0

print("Mejores parámetros:", grid_rbf.best_params_)
print(f"Mejor CV score: {grid_rbf.best_score_:.4f}")
print(f"Test score: {grid_rbf.best_estimator_.score(X_test, y_test):.4f}")
print(f"Tiempo GridSearch: {grid_time:.3f}s")


Mejores parámetros: {'svm__C': 100, 'svm__gamma': 0.01}
Mejor CV score: 0.9401
Test score: 0.9431
Tiempo GridSearch: 3.725s


In [8]:
# Tabla con al menos 10 combinaciones representativas: usamos las 10 mejores por CV
cvres = pd.DataFrame(grid_rbf.cv_results_)
cvres = cvres.sort_values('mean_test_score', ascending=False)

top = cvres.head(10).copy()
rows = []
for _, r in top.iterrows():
    C = r['param_svm__C']
    gamma = r['param_svm__gamma']

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', C=C, gamma=gamma, random_state=RANDOM_STATE))
    ])
    t0 = perf_counter()
    pipe.fit(X_train, y_train)
    t_fit = perf_counter() - t0

    y_pred = pipe.predict(X_test)
    n_sv = int(pipe.named_steps['svm'].n_support_.sum())
    rows.append({
        'C': float(C),
        'gamma': str(gamma),
        'CV Accuracy': float(r['mean_test_score']),
        'Test Accuracy': accuracy_score(y_test, y_pred),
        'Nº SV': n_sv,
        '% Train': 100 * n_sv / len(X_train),
        'Tiempo (s)': t_fit
    })

df_svm_rbf = pd.DataFrame(rows)
df_svm_rbf


,C,gamma,CV Accuracy,Test Accuracy,Nº SV,% Train,Tiempo (s)
0,100.0,0.01,0.940059,0.943128,629,37.329377,0.042572
1,100.0,0.001,0.922255,0.940758,879,52.166172,0.047493
2,100.0,scale,0.908605,0.921801,799,47.418398,0.047326
3,100.0,auto,0.908605,0.921801,799,47.418398,0.045705
4,10.0,scale,0.908012,0.909953,850,50.445104,0.046181
5,10.0,auto,0.908012,0.909953,850,50.445104,0.047440
6,10.0,0.01,0.907418,0.919431,880,52.225519,0.040854
7,10.0,0.1,0.889614,0.898104,943,55.964392,0.064266
8,100.0,0.1,0.889021,0.902844,939,55.727003,0.063836
9,1.0,0.1,0.865875,0.860190,1053,62.492582,0.060063


In [9]:
save_table(df_svm_rbf, 'svm_rbf_grid')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/svm_rbf_grid.tex')

La mejor combinación encontrada es **C = 100** y **γ = 0.01** (CV = 0.9401, Test = 0.9431), superando al kernel RBF por defecto en casi 9 puntos porcentuales. El valor alto de C indica que el modelo se beneficia de reducir la tolerancia a errores de clasificación, mientras que γ = 0.01 mantiene una influencia moderada de cada muestra, evitando *overfitting* con fronteras demasiado irregulares.

### 3.3 Optimización del kernel polinomial (GridSearchCV)

El kernel polinomial (`(γ·⟨x, x'⟩ + r)^d`) permite controlar la complejidad de la frontera mediante el grado (degree) y la combinación de C y γ. Exploramos degree ∈ {2, 3, 4}, C ∈ {0.1, 1, 10} y γ ∈ {'scale', 'auto', 0.1}.

In [10]:
pipe_poly = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='poly', random_state=RANDOM_STATE))
])

param_grid_poly = {
    'svm__degree': [2, 3, 4],
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 'auto', 0.1]
}

grid_poly = GridSearchCV(
    pipe_poly, param_grid_poly,
    cv=5, scoring='accuracy', n_jobs=-1, refit=True
)

t0 = perf_counter()
grid_poly.fit(X_train, y_train)
grid_time_poly = perf_counter() - t0

print("Mejores parámetros:", grid_poly.best_params_)
print(f"Mejor CV score: {grid_poly.best_score_:.4f}")
print(f"Test score: {grid_poly.best_estimator_.score(X_test, y_test):.4f}")
print(f"Tiempo GridSearch: {grid_time_poly:.3f}s")


Mejores parámetros: {'svm__C': 10, 'svm__degree': 3, 'svm__gamma': 0.1}
Mejor CV score: 0.8718
Test score: 0.8886
Tiempo GridSearch: 0.901s


In [11]:
cvres_poly = pd.DataFrame(grid_poly.cv_results_).sort_values('mean_test_score', ascending=False)

# Cogemos 10 representativas (top 10)
top = cvres_poly.head(10).copy()
rows = []
for _, r in top.iterrows():
    degree = int(r['param_svm__degree'])
    C = float(r['param_svm__C'])
    gamma = r['param_svm__gamma']

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='poly', degree=degree, C=C, gamma=gamma, random_state=RANDOM_STATE))
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    n_sv = int(pipe.named_steps['svm'].n_support_.sum())
    rows.append({
        'degree': degree,
        'C': C,
        'gamma': str(gamma),
        'CV Accuracy': float(r['mean_test_score']),
        'Test Accuracy': accuracy_score(y_test, y_pred),
        'Nº SV': n_sv
    })

df_svm_poly = pd.DataFrame(rows)
df_svm_poly


,degree,C,gamma,CV Accuracy,Test Accuracy,Nº SV
0,3,10.0,0.1,0.871810,0.888626,789
1,3,1.0,0.1,0.864095,0.876777,897
2,3,10.0,auto,0.860534,0.879147,916
3,3,10.0,scale,0.860534,0.879147,916
4,2,10.0,0.1,0.858754,0.845972,783
5,2,10.0,scale,0.847478,0.855450,900
6,2,10.0,auto,0.847478,0.855450,900
7,4,1.0,0.1,0.839169,0.843602,959
8,4,10.0,0.1,0.835608,0.845972,856
9,2,1.0,0.1,0.830861,0.838863,946


In [12]:
save_table(df_svm_poly, 'svm_poly_grid')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/svm_poly_grid.tex')

La mejor configuración polinomial es **degree = 3, C = 10, γ = 0.1** (CV = 0.8718, Test = 0.8886). Aunque mejora respecto al kernel polinomial por defecto, se queda por debajo tanto del kernel lineal como del RBF optimizado. Grados mayores (d = 4) no mejoran el rendimiento, lo que sugiere que la relación entre las features no requiere interacciones de orden superior para este problema.

### 3.4 Comparación de implementaciones lineales (SVC linear vs LinearSVC vs SGDClassifier)

Scikit-learn ofrece tres implementaciones del caso lineal con distinta formulación interna y coste computacional:

| Implementación | Formulación | Uso recomendado |
|:---|:---|:---|
| `SVC(kernel='linear')` | Dual (libsvm) | n_samples < 10,000 |
| `LinearSVC` | Primal (liblinear) | Datasets medianos, kernel lineal |
| `SGDClassifier(loss='hinge')` | SGD con hinge loss | Datasets grandes (> 100,000) |

Comparamos rendimiento y tiempo de entrenamiento de las tres implementaciones.

In [13]:
models_linear = {
    'SVC(linear)':   Pipeline([('scaler', StandardScaler()),
                               ('svm', SVC(kernel='linear', random_state=RANDOM_STATE))]),
    'LinearSVC':     Pipeline([('scaler', StandardScaler()),
                               ('svm', LinearSVC(random_state=RANDOM_STATE, max_iter=10000))]),
    'SGDClassifier': Pipeline([('scaler', StandardScaler()),
                               ('svm', SGDClassifier(loss='hinge', random_state=RANDOM_STATE))]),
}

rows = []
for name, model in models_linear.items():
    t0 = perf_counter()
    model.fit(X_train, y_train)
    fit_time = perf_counter() - t0

    cv = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    y_pred = model.predict(X_test)

    n_sv = '-'
    if name.startswith('SVC'):
        n_sv = int(model.named_steps['svm'].n_support_.sum())

    rows.append({
        'Implementación': name,
        'CV Accuracy': cv.mean(),
        'Test Accuracy': accuracy_score(y_test, y_pred),
        'Nº SV': n_sv,
        'Tiempo (s)': fit_time
    })

df_svm_linear_impl = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
df_svm_linear_impl


,Implementación,CV Accuracy,Test Accuracy,Nº SV,Tiempo (s)
0,SVC(linear),0.941840,0.938389,565,0.058051
1,LinearSVC,0.763205,0.786730,-,0.047093
2,SGDClassifier,0.706231,0.694313,-,0.027383


In [14]:
save_table(df_svm_linear_impl, 'svm_linear_impl')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/svm_linear_impl.tex')

`SVC(linear)` es claramente superior en accuracy (CV = 0.9418, Test = 0.9384), mientras que `LinearSVC` y `SGDClassifier` quedan por debajo (CV ≈ 0.76 y 0.71 respectivamente). La diferencia se debe a que `SVC` resuelve el problema dual exacto, mientras que `LinearSVC` usa otra formulación (liblinear) y `SGDClassifier` realiza optimización estocástica que puede no converger al óptimo global sin un ajuste cuidadoso de hiperparámetros. En tiempo, las tres son similares dado el tamaño reducido de nuestro dataset (~1685 muestras). Para datasets masivos (>100k), SGDClassifier sería la única opción viable.

### 3.5 Análisis de vectores de soporte (configuraciones representativas)

Los vectores de soporte son las muestras de entrenamiento que definen la frontera de decisión. A mayor número de SV, el modelo utiliza más datos para trazar la frontera, lo que puede indicar mayor complejidad o mayor dificultad del problema para ese kernel. Comparamos el número de SV en distintas configuraciones.

In [15]:
configs = [
    ('Linear (C=1)',        dict(kernel='linear', C=1, gamma='scale')),
    ('RBF (C=1, γ=scale)',  dict(kernel='rbf', C=1, gamma='scale')),
    ('RBF (C=10, γ=0.1)',   dict(kernel='rbf', C=10, gamma=0.1)),
    ('RBF (C=100, γ=1)',    dict(kernel='rbf', C=100, gamma=1)),
    ('Poly (d=3, C=1)',     dict(kernel='poly', degree=3, C=1, gamma='scale')),
]

rows = []
for name, params in configs:
    svc = SVC(random_state=RANDOM_STATE, **params)
    pipe = Pipeline([('scaler', StandardScaler()), ('svm', svc)])
    pipe.fit(X_train, y_train)

    n_sv = int(pipe.named_steps['svm'].n_support_.sum())
    rows.append({
        'Configuración': name,
        'Nº SV': n_sv,
        '% Training Set': 100 * n_sv / len(X_train),
    })

df_sv = pd.DataFrame(rows).sort_values('% Training Set')
df_sv


,Configuración,Nº SV,% Training Set
0,Linear (C=1),565,33.531157
2,"RBF (C=10, γ=0.1)",943,55.964392
1,"RBF (C=1, γ=scale)",1089,64.629080
4,"Poly (d=3, C=1)",1202,71.335312
3,"RBF (C=100, γ=1)",1260,74.777448


In [16]:
save_table(df_sv, 'svm_support_vectors')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/svm_support_vectors.tex')

La configuración con menor porcentaje de SV es el kernel lineal (C = 1), lo que indica que necesita relativamente pocos puntos de referencia para definir la frontera. La configuración extrema **RBF (C = 100, γ = 1)** usa una proporción elevada del training set como SV, señal de que con γ alto la frontera es tan local que prácticamente memoriza los datos (riesgo de *overfitting*).

### 3.6 Evaluación final SVM (mejor modelo) + matriz de confusión

Evaluamos el mejor modelo SVM encontrado (RBF con C = 100, γ = 0.01) en el conjunto de test y generamos su *classification report* y matriz de confusión.

In [17]:
best_svm = grid_rbf.best_estimator_
best_svm.fit(X_train, y_train)
y_pred_svm = best_svm.predict(X_test)

print(classification_report(y_test, y_pred_svm, target_names=class_names))

plot_confusion(
    y_test, y_pred_svm, class_names,
    filename='confusion_matrix_svm.png',
    title='Matriz de Confusión — Mejor SVM'
)


                     precision    recall  f1-score   support

Insufficient_Weight       0.96      0.98      0.97        54
      Normal_Weight       0.86      0.86      0.86        57
     Obesity_Type_I       0.92      0.99      0.95        70
    Obesity_Type_II       0.98      0.93      0.96        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0.90      0.90      0.90        58
Overweight_Level_II       0.98      0.95      0.96        58

           accuracy                           0.94       422
          macro avg       0.94      0.94      0.94       422
       weighted avg       0.94      0.94      0.94       422



In [18]:
# Tabla resumen (Parte 1)
def summarize_model(name, model, is_svm=True):
    cv = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    m = metrics_row(y_test, y_pred)
    n_sv = '-'
    pct = '-'
    if is_svm and hasattr(model.named_steps['svm'], 'n_support_'):
        n_sv = int(model.named_steps['svm'].n_support_.sum())
        pct = 100 * n_sv / len(X_train)
    return {
        'Modelo': name,
        'CV Accuracy': cv.mean(),
        'Test Accuracy': m['Test Accuracy'],
        'F1 (w)': m['F1 (w)'],
        'Nº SV': n_sv,
        '% Train SV': pct
    }

svm_summary_rows = [
    summarize_model('SVM Linear', Pipeline([('scaler', StandardScaler()),
                                           ('svm', SVC(kernel='linear', random_state=RANDOM_STATE))]), True),
    summarize_model('SVM Poly (best)', grid_poly.best_estimator_, True),
    summarize_model('SVM RBF (best)', grid_rbf.best_estimator_, True),
    summarize_model('SVM Sigmoid', Pipeline([('scaler', StandardScaler()),
                                            ('svm', SVC(kernel='sigmoid', random_state=RANDOM_STATE))]), True),
]

df_svm_summary = pd.DataFrame(svm_summary_rows).sort_values('CV Accuracy', ascending=False)
df_svm_summary


,Modelo,CV Accuracy,Test Accuracy,F1 (w),Nº SV,% Train SV
0,SVM Linear,0.941840,0.938389,0.938064,565,33.531157
2,SVM RBF (best),0.940059,0.943128,0.943202,629,37.329377
1,SVM Poly (best),0.871810,0.888626,0.888466,789,46.824926
3,SVM Sigmoid,0.640950,0.651659,0.650088,1217,72.225519


In [19]:
save_table(df_svm_summary, 'svm_summary')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/svm_summary.tex')

El mejor SVM (RBF, C = 100, γ = 0.01) alcanza un **Test Accuracy de 0.9431** y un F1 ponderado de 0.94. Las clases extremas (Insufficient_Weight, Obesity_Type_III) obtienen F1 > 0.95, mientras que las intermedias (Normal_Weight, Overweight_Level_I) se benefician de la frontera no lineal para mejorar respecto a la Práctica 0 con Regresión Logística. El kernel lineal queda muy cerca (0.9384), lo que confirma que nuestros datos son bastante separables en el espacio original de 23 dimensiones.

La tabla resumen de la Parte 1 recoge los mejores resultados por cada familia de kernel.

---

## 4. Parte 2 — Árboles de Decisión

Los árboles de decisión (CART) realizan particiones recursivas del espacio de features según criterios de impureza (Gini o entropía). Su principal ventaja es la interpretabilidad: producen reglas de decisión legibles. El reto principal es controlar el *overfitting*, ya que sin restricciones un árbol puede memorizar completamente el training set. Para combatirlo se usan dos estrategias:

- **Poda previa (pre-pruning):** limitar la profundidad (`max_depth`), el mínimo de muestras por split/hoja o el número de hojas.
- **Poda posterior (post-pruning, CCP):** entrenar el árbol completo y después ir podando ramas usando el parámetro `ccp_alpha` (cost-complexity pruning).

### 4.1 Árbol sin restricciones (baseline)

Empezamos por entrenar un árbol sin ninguna restricción para observar el comportamiento de *overfitting*: esperamos que alcance un 100% de accuracy en train pero un rendimiento inferior en test.

In [20]:
tree_full = DecisionTreeClassifier(random_state=RANDOM_STATE)
t0 = perf_counter()
tree_full.fit(X_train, y_train)
t_fit = perf_counter() - t0

train_acc = tree_full.score(X_train, y_train)
test_acc = tree_full.score(X_test, y_test)

df_tree_baseline = pd.DataFrame([{
    'Configuración': 'Sin restricciones',
    'Train Accuracy': train_acc,
    'Test Accuracy': test_acc,
    'Profundidad': tree_full.get_depth(),
    'Nº Hojas': tree_full.get_n_leaves(),
    'Tiempo (s)': t_fit
}])
df_tree_baseline


,Configuración,Train Accuracy,Test Accuracy,Profundidad,Nº Hojas,Tiempo (s)
0,Sin restricciones,1.0,0.92891,12,101,0.008541


In [21]:
save_table(df_tree_baseline, 'tree_baseline')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_baseline.tex')

Efectivamente, el árbol sin restricciones alcanza **100% en train** pero solo **92.89% en test**, con una profundidad de 12 niveles y 101 hojas. La brecha de ~7 puntos entre train y test es señal clara de *overfitting*: el modelo ha memorizado el ruido del training set, creando hojas que responden a patrones no generalizables.

### 4.2 Efecto de max_depth

Exploramos cómo limitar la profundidad máxima del árbol afecta al equilibrio entre sesgo y varianza. A poca profundidad (1–3), el modelo es demasiado simple (*underfitting*); a profundidad sin límite, se produce *overfitting*. El objetivo es encontrar el punto intermedio.

In [22]:
depths = [1, 2, 3, 5, 7, 10, 15, None]
rows = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_STATE)

    cv_scores = cross_val_score(tree, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()

    tree.fit(X_train, y_train)
    rows.append({
        'max_depth': 'None' if depth is None else depth,
        'Train Acc': tree.score(X_train, y_train),
        'CV Acc': cv_mean,
        'Test Acc': tree.score(X_test, y_test),
        'Profundidad real': tree.get_depth(),
        'Nº Hojas': tree.get_n_leaves()
    })

df_tree_depth = pd.DataFrame(rows)
df_tree_depth


,max_depth,Train Acc,CV Acc,Test Acc,Profundidad real,Nº Hojas
0,1,0.290801,0.290801,0.291469,1,2
1,2,0.554303,0.553116,0.545024,2,4
2,3,0.643323,0.629080,0.656398,3,8
3,5,0.852819,0.823145,0.819905,5,25
4,7,0.945401,0.885460,0.900474,7,57
5,10,0.998813,0.925223,0.928910,10,98
6,15,1.000000,0.922849,0.928910,12,101
7,None,1.000000,0.924036,0.928910,12,101


In [23]:
# Curva profundidad vs rendimiento
dfp = df_tree_depth.copy()
dfp['max_depth_num'] = dfp['max_depth'].replace('None', np.nan).astype(float)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dfp['max_depth_num'], dfp['Train Acc'], marker='o', label='Train')
ax.plot(dfp['max_depth_num'], dfp['CV Acc'], marker='o', label='CV')
ax.plot(dfp['max_depth_num'], dfp['Test Acc'], marker='o', label='Test')
ax.set_xlabel('max_depth (None omitido)')
ax.set_ylabel('Accuracy')
ax.set_title('Efecto de max_depth en Decision Tree')
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(IMAGES_DIR / 'decision_tree_depth_curve.png')


C:\Users\jordi\AppData\Local\Temp\ipykernel_23876\1536526240.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfp['max_depth_num'] = dfp['max_depth'].replace('None', np.nan).astype(float)


In [24]:
save_table(df_tree_depth, 'tree_max_depth')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_max_depth.tex')

La gráfica y la tabla muestran el patrón clásico: con **max_depth = 1** (stump) el accuracy de CV es muy bajo porque un solo split no puede capturar la complejidad de 7 clases. A partir de **max_depth = 5–7** el rendimiento se estabiliza, y aumentar la profundidad más allá no mejora la generalización sino que incrementa el *overfitting* (train sube pero CV se estanca o baja). El test accuracy máximo se obtiene en torno a profundidad 7–10.

### 4.3 Criterios de división

Comparamos los tres criterios de impureza disponibles para clasificación: **Gini** (usado por defecto en CART), **Entropía** (ID3/C4.5) y **Log Loss**. Los tres miden la heterogeneidad de los nodos, pero con fórmulas ligeramente distintas.

In [25]:
criterios = ['gini', 'entropy', 'log_loss']
rows = []
for crit in criterios:
    tree = DecisionTreeClassifier(criterion=crit, random_state=RANDOM_STATE)
    cv_scores = cross_val_score(tree, X_train, y_train, cv=5, scoring='accuracy')

    tree.fit(X_train, y_train)
    rows.append({
        'Criterio': crit,
        'CV Accuracy': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'Test Accuracy': tree.score(X_test, y_test),
        'Nº Hojas': tree.get_n_leaves()
    })

df_tree_criteria = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
df_tree_criteria


,Criterio,CV Accuracy,CV Std,Test Accuracy,Nº Hojas
1,entropy,0.941840,0.011508,0.959716,79
2,log_loss,0.941840,0.011508,0.959716,79
0,gini,0.924036,0.011039,0.928910,101


In [26]:
save_table(df_tree_criteria[['Criterio','CV Accuracy','Test Accuracy','Nº Hojas']], 'tree_criteria')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_criteria.tex')

Los tres criterios producen resultados muy similares, con diferencias menores a 1 punto porcentual. Es un resultado habitual: la teoría predice que Gini y Entropía difieren poco en la práctica, y log_loss es equivalente a entropía para clasificación. La elección del criterio es, en este caso, irrelevante frente a otros hiperparámetros como la profundidad o el mínimo de muestras por hoja.

### 4.4 Poda previa — GridSearchCV

Además de `max_depth`, existen otros parámetros de poda previa: `min_samples_split` (mínimo de muestras para dividir un nodo), `min_samples_leaf` (mínimo de muestras en cada hoja) y `max_leaf_nodes` (máximo número de hojas). Exploramos combinaciones de todos ellos con `GridSearchCV`.

In [27]:
param_grid_tree = {
    'max_depth':         [3, 5, 10, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf':  [1, 2, 5, 10],
    'max_leaf_nodes':    [None, 10, 20, 50]
}

grid_tree = GridSearchCV(
    DecisionTreeClassifier(random_state=RANDOM_STATE),
    param_grid_tree,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    refit=True
)

t0 = perf_counter()
grid_tree.fit(X_train, y_train)
t_grid = perf_counter() - t0

print("Mejores parámetros:", grid_tree.best_params_)
print(f"Mejor CV score: {grid_tree.best_score_:.4f}")
print(f"Test score: {grid_tree.best_estimator_.score(X_test, y_test):.4f}")
print(f"Tiempo GridSearch: {t_grid:.3f}s")


Mejores parámetros: {'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mejor CV score: 0.9288
Test score: 0.9218
Tiempo GridSearch: 0.976s


In [28]:
cvres_tree = pd.DataFrame(grid_tree.cv_results_).sort_values('mean_test_score', ascending=False)
top = cvres_tree.head(10).copy()

rows = []
for _, r in top.iterrows():
    params = {
        'max_depth': r['param_max_depth'],
        'min_samples_split': int(r['param_min_samples_split']),
        'min_samples_leaf': int(r['param_min_samples_leaf']),
        'max_leaf_nodes': r['param_max_leaf_nodes'],
    }
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, **params)
    model.fit(X_train, y_train)
    rows.append({
        'max_depth': 'None' if params['max_depth'] is None else int(params['max_depth']),
        'min_samples_split': params['min_samples_split'],
        'min_samples_leaf': params['min_samples_leaf'],
        'max_leaf_nodes': 'None' if params['max_leaf_nodes'] is None else int(params['max_leaf_nodes']),
        'CV Acc': float(r['mean_test_score']),
        'Test Acc': model.score(X_test, y_test),
    })

df_tree_grid = pd.DataFrame(rows)
df_tree_grid


,max_depth,min_samples_split,min_samples_leaf,max_leaf_nodes,CV Acc,Test Acc
0,None,2,2,None,0.928783,0.921801
1,None,5,1,None,0.927596,0.928910
2,None,5,2,None,0.927003,0.917062
3,10,2,2,None,0.925223,0.921801
4,10,2,1,None,0.925223,0.928910
5,10,5,1,None,0.925223,0.928910
6,10,5,2,None,0.924036,0.921801
7,None,2,1,None,0.924036,0.928910
8,None,2,2,50,0.922849,0.926540
9,10,2,2,50,0.922255,0.926540


In [29]:
save_table(df_tree_grid, 'tree_grid_prepruning')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_grid_prepruning.tex')

La mejor configuración de poda previa encontrada por GridSearch es **max_depth = None, min_samples_leaf = 2, min_samples_split = 2, max_leaf_nodes = None** (CV = 0.9288, Test = 0.9218). El parámetro clave es `min_samples_leaf = 2`, que impide que se creen hojas con una sola muestra, reduciendo el sobreajuste sin limitar la profundidad.

### 4.5 Poda posterior — Cost-Complexity Pruning (ccp_alpha)

CART implementa poda posterior mediante el parámetro `ccp_alpha`. A mayor valor de alpha, mayor poda: se eliminan las ramas cuya contribución a la reducción de impureza no compense el incremento de complejidad. El procedimiento es: (1) entrenar el árbol completo, (2) obtener la secuencia de alphas mediante `cost_complexity_pruning_path`, y (3) evaluar cada alpha con CV para encontrar el equilibrio óptimo.

In [30]:
path = DecisionTreeClassifier(random_state=RANDOM_STATE).cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas[:-1]  # último -> solo raíz

# Selección de alphas representativos (si hay muchos)
if len(ccp_alphas) > 12:
    idx = np.unique(np.round(np.linspace(0, len(ccp_alphas)-1, 12)).astype(int))
    alphas_sel = ccp_alphas[idx]
else:
    alphas_sel = ccp_alphas

rows = []
best = {'alpha': None, 'cv': -1}

for alpha in alphas_sel:
    tree = DecisionTreeClassifier(ccp_alpha=float(alpha), random_state=RANDOM_STATE)
    cv_scores = cross_val_score(tree, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()

    tree.fit(X_train, y_train)
    test_acc = tree.score(X_test, y_test)

    rows.append({
        'ccp_alpha': float(alpha),
        'CV Accuracy': cv_mean,
        'Test Accuracy': test_acc,
        'Profundidad': tree.get_depth(),
        'Nº Hojas': tree.get_n_leaves()
    })
    if cv_mean > best['cv']:
        best = {'alpha': float(alpha), 'cv': cv_mean}

df_tree_ccp = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
print('Mejor alpha (en los seleccionados):', best)
df_tree_ccp


Mejor alpha (en los seleccionados): {'alpha': 0.000791295746785361, 'cv': np.float64(0.9275964391691394)}


,ccp_alpha,CV Accuracy,Test Accuracy,Profundidad,Nº Hojas
2,0.000791,0.927596,0.928910,10,74
4,0.001149,0.926409,0.926540,10,57
5,0.001385,0.925223,0.924171,10,49
3,0.001107,0.925223,0.926540,10,67
0,0.000000,0.924036,0.928910,12,101
1,0.000581,0.924036,0.928910,10,86
6,0.002148,0.918101,0.936019,9,41
7,0.002428,0.915134,0.928910,8,35
8,0.004285,0.899110,0.924171,8,28
9,0.008398,0.872404,0.900474,8,22


In [31]:
# Curva alpha vs rendimiento (sobre los alphas seleccionados)
dfp = df_tree_ccp.sort_values('ccp_alpha')
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(dfp['ccp_alpha'], dfp['CV Accuracy'], marker='o', label='CV')
ax.plot(dfp['ccp_alpha'], dfp['Test Accuracy'], marker='o', label='Test')
ax.set_xlabel('ccp_alpha')
ax.set_ylabel('Accuracy')
ax.set_title('Cost-Complexity Pruning: alpha vs rendimiento')
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(IMAGES_DIR / 'decision_tree_ccp_curve.png')


In [32]:
save_table(df_tree_ccp, 'tree_ccp')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_ccp.tex')

El mejor `ccp_alpha ≈ 0.00079` produce un CV de 0.9276, con una profundidad de 10 y 74 hojas. La curva muestra cómo, a medida que alpha crece, el árbol se simplifica: al principio la poda elimina ramas ruidosas mejorando la generalización, pero si alpha es demasiado grande el árbol se poda en exceso y pierde capacidad discriminativa.

### 4.6 Importancia de características (mejor árbol por poda previa)

Una de las grandes ventajas de los árboles de decisión es que permiten calcular la **importancia de cada feature** según la reducción media de impureza (Gini importance). Las variables usadas en los primeros splits del árbol tienen mayor importancia.

In [33]:
best_tree = grid_tree.best_estimator_
best_tree.fit(X_train, y_train)

importances = best_tree.feature_importances_
feature_names = X_train.columns.tolist()

df_tree_imp = pd.DataFrame({
    'Feature': feature_names,
    'Importancia (Gini)': importances
}).sort_values('Importancia (Gini)', ascending=False)

df_tree_imp.head(15)


,Feature,Importancia (Gini)
3,Weight,0.506558
2,Height,0.246968
0,Gender,0.160277
1,Age,0.024752
5,FAVC,0.023797
9,CH2O,0.010139
14,CAEC_Sometimes,0.007591
12,TUE,0.004725
7,NCP,0.003756
13,CAEC_Frequently,0.002327


In [34]:
# Plot top-15 importancias
topk = df_tree_imp.head(15).iloc[::-1]
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(topk['Feature'], topk['Importancia (Gini)'])
ax.set_xlabel('Importancia')
ax.set_title('Decision Tree — Importancia de características (top 15)')
save_fig(IMAGES_DIR / 'decision_tree_feature_importance.png')


In [35]:
save_table(df_tree_imp.head(20), 'tree_feature_importance')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_feature_importance.tex')

Como era de esperar, **Weight** y **Height** dominan ampliamente la importancia, coherente con lo que observamos en los scatter plots de la Práctica 0 (el IMC se calcula directamente a partir de estas dos variables). El resto de features tienen importancias mucho menores pero contribuyen colectivamente a las decisiones en niveles más profundos del árbol.

### 4.7 Visualización del árbol (profundidad limitada para legibilidad)

Visualizamos los primeros 4 niveles del mejor árbol. Se puede observar cómo las primeras divisiones utilizan Weight para separar rápidamente los niveles extremos de obesidad, y luego Height refina las categorías intermedias.

In [36]:
fig, ax = plt.subplots(figsize=(22, 10))
plot_tree(
    best_tree,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,
    rounded=True,
    fontsize=8,
    max_depth=4,
    ax=ax
)
save_fig(IMAGES_DIR / 'decision_tree.png', dpi=250)

print(export_text(best_tree, feature_names=feature_names, max_depth=4))


|--- Weight <= 99.54
|   |--- Weight <= 60.06
|   |   |--- Height <= 1.66
|   |   |   |--- Weight <= 46.83
|   |   |   |   |--- Height <= 1.51
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- Height >  1.51
|   |   |   |   |   |--- truncated branch of depth 3
|   |   |   |--- Weight >  46.83
|   |   |   |   |--- Height <= 1.52
|   |   |   |   |   |--- truncated branch of depth 3
|   |   |   |   |--- Height >  1.52
|   |   |   |   |   |--- truncated branch of depth 4
|   |   |--- Height >  1.66
|   |   |   |--- Weight <= 60.00
|   |   |   |   |--- TUE <= 0.02
|   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |--- TUE >  0.02
|   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |--- Weight >  60.00
|   |   |   |   |--- Height <= 1.82
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- Height >  1.82
|   |   |   |   |   |--- class: 0
|   |--- Weight >  60.06
|   |   |--- Weight <= 76.04
|   |   |   |--- Height <= 1.72
|   |   |   |   |--- FAVC <= 0.

### 4.8 Evaluación final (Parte 2)

Comparamos las tres estrategias de árboles de decisión:

- **Sin restricciones**: árbol completo sin ninguna limitación de crecimiento.
- **Mejor poda previa (pre-pruning)**: mejor combinación de `max_depth`, `min_samples_split` y `min_samples_leaf` obtenida por GridSearchCV.
- **Mejor poda posterior (post-pruning)**: mejor `ccp_alpha` seleccionado por validación cruzada.

En la tabla reportamos Test Accuracy, F1 ponderado, profundidad y número de hojas para cada variante.

In [37]:
# Mejor árbol por poda posterior (sobre los alphas seleccionados)
alpha_best = df_tree_ccp.sort_values('CV Accuracy', ascending=False).iloc[0]['ccp_alpha']
tree_ccp_best = DecisionTreeClassifier(ccp_alpha=float(alpha_best), random_state=RANDOM_STATE)

# Fit/eval
tree_ccp_best.fit(X_train, y_train)
y_pred_ccp = tree_ccp_best.predict(X_test)

tree_pre_best = grid_tree.best_estimator_
tree_pre_best.fit(X_train, y_train)
y_pred_pre = tree_pre_best.predict(X_test)

y_pred_full = tree_full.predict(X_test)

df_tree_summary = pd.DataFrame([
    {'Modelo': 'Sin restricciones', **metrics_row(y_test, y_pred_full),
     'Profundidad': tree_full.get_depth(), 'Nº Hojas': tree_full.get_n_leaves()},
    {'Modelo': 'Mejor poda previa', **metrics_row(y_test, y_pred_pre),
     'Profundidad': tree_pre_best.get_depth(), 'Nº Hojas': tree_pre_best.get_n_leaves()},
    {'Modelo': 'Mejor poda posterior', **metrics_row(y_test, y_pred_ccp),
     'Profundidad': tree_ccp_best.get_depth(), 'Nº Hojas': tree_ccp_best.get_n_leaves()},
]).sort_values('F1 (w)', ascending=False)

df_tree_summary


,Modelo,Test Accuracy,Precision (w),Recall (w),F1 (w),Profundidad,Nº Hojas
0,Sin restricciones,0.928910,0.932683,0.928910,0.929719,12,101
2,Mejor poda posterior,0.928910,0.932683,0.928910,0.929719,10,74
1,Mejor poda previa,0.921801,0.925161,0.921801,0.922354,11,90


In [38]:
save_table(df_tree_summary, 'tree_summary')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/tree_summary.tex')

In [39]:
best_tree_global = tree_pre_best  # por defecto, usamos el mejor pre-pruning
y_pred_best_tree = best_tree_global.predict(X_test)
plot_confusion(
    y_test, y_pred_best_tree, class_names,
    filename='confusion_matrix_tree.png',
    title='Matriz de Confusión — Mejor Decision Tree'
)


**Análisis de resultados (Parte 2):**

El árbol **sin restricciones** y la **poda posterior** obtienen el mismo Test Accuracy (92.89 %), pero la poda posterior logra resultados equivalentes con un árbol **más compacto** (profundidad 10, 74 hojas frente a profundidad 12, 101 hojas). La poda previa queda ligeramente por debajo (92.18 %), probablemente porque las restricciones de `min_samples_leaf` impiden capturar algunas divisiones finas que sí se modelan con el árbol completo.

En general, los árboles de decisión individuales alcanzan un techo en torno al 93 % de accuracy en este dataset. La matriz de confusión revela que las categorías intermedias (Normal_Weight, Overweight_Level_I/II) son las más propensas a confusión, lo cual tiene sentido dado que comparten rangos de IMC cercanos.

---

## 5. Parte 3 — Random Forest y Extra Trees

En esta sección pasamos de un único árbol de decisión a **métodos de ensemble** basados en *bagging*. La idea central de Random Forest es entrenar múltiples árboles de decisión sobre subconjuntos aleatorios de los datos (bootstrap) y de las features, y agregar sus predicciones por votación mayoritaria. Esta diversificación reduce la varianza del modelo sin aumentar significativamente el sesgo, lo que suele mejorar la generalización respecto a un árbol individual.

Además de Random Forest, evaluaremos **Extra Trees** (Extremely Randomized Trees), una variante que introduce aún más aleatoriedad al elegir los umbrales de corte de forma aleatoria en lugar de buscar los óptimos.

### 5.1 Random Forest por defecto

Comenzamos entrenando un Random Forest con la configuración por defecto de scikit-learn (`n_estimators=100`, `max_features='sqrt'`), activando el **OOB score** (Out-of-Bag) como estimación del error de generalización sin necesidad de validación cruzada. Cada árbol se entrena sobre una muestra bootstrap (~63.2 % de los datos), y el OOB score se calcula evaluando cada muestra solo con los árboles que **no** la incluyeron en su bootstrap.

In [40]:
rf_default = RandomForestClassifier(
    n_estimators=100, random_state=RANDOM_STATE,
    n_jobs=-1, oob_score=True
)
t0 = perf_counter()
rf_default.fit(X_train, y_train)
t_fit = perf_counter() - t0

y_pred = rf_default.predict(X_test)
m = metrics_row(y_test, y_pred)

df_rf_default = pd.DataFrame([{
    'Configuración': 'RF por defecto (n=100)',
    'OOB Score': rf_default.oob_score_,
    'Test Accuracy': m['Test Accuracy'],
    'F1 (w)': m['F1 (w)'],
    'Tiempo (s)': t_fit
}])
df_rf_default


,Configuración,OOB Score,Test Accuracy,F1 (w),Tiempo (s)
0,RF por defecto (n=100),0.941246,0.945498,0.946826,0.138665


In [41]:
save_table(df_rf_default, 'rf_default')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/rf_default.tex')

### 5.2 Efecto del número de estimadores

El parámetro `n_estimators` controla cuántos árboles se combinan en el ensemble. Más árboles proporcionan predicciones más estables (menor varianza), pero con rendimientos decrecientes y mayor coste computacional. Entrenamos Random Forest con 10, 25, 50, 100, 200 y 500 árboles para observar la curva de convergencia tanto del OOB score como de la accuracy en test.

In [42]:
rows = []
for n in [10, 25, 50, 100, 200, 500]:
    rf = RandomForestClassifier(
        n_estimators=n, random_state=RANDOM_STATE, n_jobs=-1, oob_score=True
    )
    t0 = perf_counter()
    rf.fit(X_train, y_train)
    t_fit = perf_counter() - t0

    rows.append({
        'n_estimators': n,
        'OOB Score': rf.oob_score_,
        'Test Accuracy': rf.score(X_test, y_test),
        'Tiempo (s)': t_fit
    })

df_rf_n = pd.DataFrame(rows)
df_rf_n


C:\Users\jordi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\ensemble\_forest.py:612: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


,n_estimators,OOB Score,Test Accuracy,Tiempo (s)
0,10,0.846884,0.909953,0.036479
1,25,0.908605,0.943128,0.051969
2,50,0.927596,0.947867,0.074461
3,100,0.941246,0.945498,0.149437
4,200,0.944214,0.947867,0.270960
5,500,0.947181,0.952607,0.626438


In [43]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_rf_n['n_estimators'], df_rf_n['OOB Score'], marker='o', label='OOB')
ax.plot(df_rf_n['n_estimators'], df_rf_n['Test Accuracy'], marker='o', label='Test')
ax.set_xlabel('n_estimators')
ax.set_ylabel('Score')
ax.set_title('Random Forest — Efecto de n_estimators')
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(IMAGES_DIR / 'rf_n_estimators_curve.png')


In [44]:
save_table(df_rf_n, 'rf_n_estimators')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/rf_n_estimators.tex')

Se observa una mejora rápida al pasar de 10 a 50 árboles (de 91.0 % a 94.8 % en test), con **rendimientos decrecientes** a partir de 100 árboles. Con 500 árboles alcanzamos 95.3 %, pero el tiempo de entrenamiento se multiplica por ×4.5 respecto a 100 árboles. El OOB score sigue una tendencia similar y converge hacia el 94.7 %, confirmando su fiabilidad como estimador del error de generalización.

### 5.3 Efecto de `max_features`

El parámetro `max_features` determina cuántas features se consideran como candidatas en cada split. Valores más bajos (`'sqrt'`, `'log2'`) aumentan la diversidad entre árboles (decorrelación), mientras que usar todas las features (`None`) permite que cada árbol sea más preciso individualmente pero aumenta la correlación entre árboles.

Comparamos cuatro opciones: `'sqrt'` ($\approx 5$), `'log2'` ($\approx 4$), `0.5` (la mitad = 11) y `None` (todas = 23).

In [45]:
rows = []
for mf in ['sqrt', 'log2', 0.5, None]:
    rf = RandomForestClassifier(
        n_estimators=100, max_features=mf,
        random_state=RANDOM_STATE, n_jobs=-1, oob_score=True
    )
    rf.fit(X_train, y_train)
    rows.append({
        'max_features': str(mf),
        'OOB Score': rf.oob_score_,
        'Test Accuracy': rf.score(X_test, y_test),
    })

df_rf_mf = pd.DataFrame(rows).sort_values('Test Accuracy', ascending=False)
df_rf_mf


,max_features,OOB Score,Test Accuracy
2,0.5,0.959644,0.954976
3,None,0.954303,0.952607
1,log2,0.941246,0.945498
0,sqrt,0.941246,0.945498


In [46]:
save_table(df_rf_mf, 'rf_max_features')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/rf_max_features.tex')

Curiosamente, usar **la mitad de las features** (`0.5`, Test = 95.50 %) y **todas las features** (`None`, Test = 95.26 %) supera a las opciones clásicas `'sqrt'` y `'log2'` (94.55 %). Esto sugiere que nuestro dataset tiene pocas features realmente predictivas (Weight, Height) y necesitan estar disponibles en la mayoría de splits para maximizar el rendimiento.

### 5.4 Grid Search (Random Forest)

Realizamos una búsqueda exhaustiva combinando `n_estimators`, `max_features`, `max_depth` y `min_samples_leaf`. Mostramos las 10 mejores configuraciones ordenadas por CV Accuracy.

In [47]:
param_grid_rf = {
    'n_estimators':     [100, 200],
    'max_features':     ['sqrt', 'log2', None],
    'max_depth':        [None, 10, 20],
    'min_samples_leaf': [1, 2, 5]
}

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid_rf,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    refit=True
)

t0 = perf_counter()
grid_rf.fit(X_train, y_train)
t_grid = perf_counter() - t0

print("Mejores parámetros:", grid_rf.best_params_)
print(f"Mejor CV score: {grid_rf.best_score_:.4f}")
print(f"Test score: {grid_rf.best_estimator_.score(X_test, y_test):.4f}")
print(f"Tiempo GridSearch: {t_grid:.3f}s")


Mejores parámetros: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 200}
Mejor CV score: 0.9519
Test score: 0.9550
Tiempo GridSearch: 11.588s


In [48]:
cvres_rf = pd.DataFrame(grid_rf.cv_results_).sort_values('mean_test_score', ascending=False)
top = cvres_rf.head(10)

rows = []
for _, r in top.iterrows():
    params = {
        'n_estimators': int(r['param_n_estimators']),
        'max_features': r['param_max_features'],
        'max_depth': r['param_max_depth'],
        'min_samples_leaf': int(r['param_min_samples_leaf'])
    }
    model = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1, **params)
    model.fit(X_train, y_train)
    rows.append({
        'n_estimators': params['n_estimators'],
        'max_features': str(params['max_features']),
        'max_depth': 'None' if params['max_depth'] is None else int(params['max_depth']),
        'min_samples_leaf': params['min_samples_leaf'],
        'CV Acc': float(r['mean_test_score']),
        'Test Acc': model.score(X_test, y_test),
    })

df_rf_grid = pd.DataFrame(rows)
df_rf_grid


,n_estimators,max_features,max_depth,min_samples_leaf,CV Acc,Test Acc
0,200,None,None,1,0.951929,0.954976
1,200,None,20,1,0.951929,0.954976
2,100,None,20,1,0.950742,0.952607
3,100,None,None,1,0.950742,0.952607
4,200,None,10,1,0.949555,0.952607
5,200,None,None,2,0.948368,0.957346
6,200,None,20,2,0.948368,0.957346
7,100,None,10,1,0.947774,0.957346
8,100,None,20,2,0.947181,0.959716
9,100,None,None,2,0.947181,0.959716


In [49]:
save_table(df_rf_grid, 'rf_grid')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/rf_grid.tex')

### 5.5 Importancia de características (Gini vs Permutation)

Calculamos la importancia de las features de dos maneras:

1. **Gini importance** (*Mean Decrease Impurity*): suma de las reducciones de impureza Gini ponderadas por la probabilidad de alcanzar cada nodo, promediada sobre todos los árboles del bosque. Es rápida pero puede estar **sesgada hacia features de alta cardinalidad**.
2. **Permutation importance**: mide cuánto cae la accuracy al permutar aleatoriamente los valores de una feature. Es más lenta pero más **robusta** y no tiene sesgos de cardinalidad.

Comparamos ambas métricas para las 15 features más relevantes.

In [50]:
best_rf = grid_rf.best_estimator_
best_rf.fit(X_train, y_train)

# Gini importance
gini_imp = pd.Series(best_rf.feature_importances_, index=X_train.columns)

# Permutation importance (más robusta)
perm = permutation_importance(best_rf, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1)
perm_imp = pd.Series(perm.importances_mean, index=X_train.columns)
perm_std = pd.Series(perm.importances_std, index=X_train.columns)

df_rf_imp = pd.DataFrame({
    'Feature': X_train.columns,
    'Gini Importance': gini_imp.values,
    'Permutation Importance': perm_imp.values,
    'Perm Std': perm_std.values
}).sort_values('Gini Importance', ascending=False)

df_rf_imp.head(15)


,Feature,Gini Importance,Permutation Importance,Perm Std
3,Weight,0.458525,0.738389,0.018725
2,Height,0.246338,0.311611,0.015403
0,Gender,0.158342,0.172038,0.013866
1,Age,0.038769,0.058057,0.007587
5,FAVC,0.023532,0.005687,0.002172
11,FAF,0.012838,-0.002133,0.002691
6,FCVC,0.009884,-0.000474,0.002552
9,CH2O,0.008880,0.002844,0.002763
12,TUE,0.008615,0.002133,0.002893
7,NCP,0.008572,0.004502,0.002691


In [51]:
topk = df_rf_imp.head(15).iloc[::-1]
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(topk['Feature'], topk['Gini Importance'])
ax.set_xlabel('Gini Importance')
ax.set_title('Random Forest — Importancia de características (top 15)')
save_fig(IMAGES_DIR / 'rf_feature_importance_gini.png')


In [52]:
save_table(df_rf_imp.head(25), 'rf_importances')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/rf_importances.tex')

### 5.6 Extra Trees (Extremely Randomized Trees)

**Extra Trees** difiere de Random Forest en dos aspectos clave:
- No usa *bootstrap* (se entrena con todo el dataset).
- Los **umbrales de corte se eligen al azar** en lugar de buscarse óptimamente.

Esta mayor aleatoriedad puede reducir aún más la varianza, aunque a costa de mayor sesgo. Suele funcionar bien en datasets donde Random Forest ya obtiene buenos resultados, pero no siempre mejora la generalización.

In [53]:
et = ExtraTreesClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1)

cv_et = cross_val_score(et, X_train, y_train, cv=5, scoring='accuracy')
t0 = perf_counter()
et.fit(X_train, y_train)
t_fit = perf_counter() - t0

y_pred = et.predict(X_test)
m = metrics_row(y_test, y_pred)

df_et = pd.DataFrame([{
    'Modelo': 'Extra Trees (n=100)',
    'CV Accuracy': cv_et.mean(),
    'Test Accuracy': m['Test Accuracy'],
    'F1 (w)': m['F1 (w)'],
    'Tiempo (s)': t_fit
}])
df_et


,Modelo,CV Accuracy,Test Accuracy,F1 (w),Tiempo (s)
0,Extra Trees (n=100),0.91632,0.92654,0.927715,0.087105


In [54]:
save_table(df_et, 'extra_trees')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/extra_trees.tex')

### 5.7 Evaluación final (Parte 3)

Comparamos todos los modelos basados en bagging junto con el árbol de decisión baseline como referencia. Incluimos Test Accuracy, F1 ponderado y OOB Score (cuando está disponible).

In [55]:
# Resumen comparativo Parte 3
# (Incluye baseline del árbol para referencia)

tree_baseline_test = tree_full.predict(X_test)
rf_default_pred = rf_default.predict(X_test)
rf_best_pred = best_rf.predict(X_test)
et_pred = et.predict(X_test)

df_rf_summary = pd.DataFrame([
    {'Modelo': 'Decision Tree (baseline)',
     **metrics_row(y_test, tree_baseline_test),
     'OOB Score': np.nan, 'Tiempo (s)': np.nan},
    {'Modelo': 'Random Forest por defecto',
     **metrics_row(y_test, rf_default_pred),
     'OOB Score': rf_default.oob_score_, 'Tiempo (s)': np.nan},
    {'Modelo': 'Random Forest mejor config',
     **metrics_row(y_test, rf_best_pred),
     'OOB Score': getattr(best_rf, 'oob_score_', np.nan), 'Tiempo (s)': np.nan},
    {'Modelo': 'Extra Trees',
     **metrics_row(y_test, et_pred),
     'OOB Score': np.nan, 'Tiempo (s)': np.nan},
]).sort_values('F1 (w)', ascending=False)

df_rf_summary


,Modelo,Test Accuracy,Precision (w),Recall (w),F1 (w),OOB Score,Tiempo (s)
2,Random Forest mejor config,0.954976,0.956814,0.954976,0.955283,NaN,NaN
1,Random Forest por defecto,0.945498,0.952690,0.945498,0.946826,0.941246,NaN
0,Decision Tree (baseline),0.928910,0.932683,0.928910,0.929719,NaN,NaN
3,Extra Trees,0.926540,0.932466,0.926540,0.927715,NaN,NaN


In [56]:
save_table(df_rf_summary, 'rf_summary')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/rf_summary.tex')

In [57]:
# Matriz de confusión del mejor modelo de la parte (por F1)
best_part3_name = df_rf_summary.iloc[0]['Modelo']
if best_part3_name == 'Random Forest mejor config':
    y_pred_best = rf_best_pred
elif best_part3_name == 'Random Forest por defecto':
    y_pred_best = rf_default_pred
elif best_part3_name == 'Extra Trees':
    y_pred_best = et_pred
else:
    y_pred_best = tree_baseline_test

plot_confusion(
    y_test, y_pred_best, class_names,
    filename='confusion_matrix_rf.png',
    title=f'Matriz de Confusión — Mejor modelo Parte 3: {best_part3_name}'
)


**Análisis de resultados (Parte 3):**

El **Random Forest con la mejor configuración** (200 árboles, todas las features, sin límite de profundidad) alcanza un **95.50 % de Test Accuracy**, una mejora sustancial respecto al mejor árbol individual (92.89 %). Esto confirma el beneficio del ensemble: la agregación de múltiples árboles reduce la varianza y suaviza los errores individuales.

Resulta interesante que **usar todas las features** (`max_features=None`) supere a `'sqrt'` y `'log2'`. Esto podría deberse a que nuestro dataset tiene solo 23 features y las dos más predictivas (Weight, Height) necesitan estar disponibles en cada split para un rendimiento óptimo.

**Extra Trees** (92.65 %) rinde por debajo del Random Forest estándar. La aleatorización extra en los umbrales no aporta beneficio aquí, probablemente porque el dataset no es tan ruidoso como para que la reducción de varianza adicional compense el aumento de sesgo.

---

## 6. Parte 4 — Gradient Boosting y AdaBoost

A diferencia del *bagging* (Random Forest), los métodos de **boosting** construyen el ensemble de forma **secuencial**: cada nuevo estimador se entrena para corregir los errores del anterior. En esta sección evaluamos dos algoritmos de boosting:

- **Gradient Boosting** (`GradientBoostingClassifier`): ajusta cada árbol al *gradiente negativo* de la función de pérdida. Permite árboles de profundidad moderada y ofrece control fino mediante `learning_rate`, `n_estimators`, `max_depth` y `subsample`.
- **AdaBoost** (`AdaBoostClassifier`): aumenta el peso de las muestras mal clasificadas en cada iteración. Clásicamente utiliza *stumps* (árboles de profundidad 1) como estimadores base.

Veremos que Gradient Boosting es especialmente potente para este dataset, mientras que AdaBoost con stumps resulta insuficiente para un problema de 7 clases.

### 6.1 Gradient Boosting por defecto

Entrenamos un `GradientBoostingClassifier` con los parámetros por defecto de scikit-learn (`n_estimators=100`, `learning_rate=0.1`, `max_depth=3`). A diferencia de Random Forest, Gradient Boosting **no admite paralelización** (los árboles se entrenan secuencialmente), por lo que el tiempo de entrenamiento será significativamente mayor.

In [58]:
gb_default = GradientBoostingClassifier(random_state=RANDOM_STATE)
cv_gb = cross_val_score(gb_default, X_train, y_train, cv=5, scoring='accuracy')

t0 = perf_counter()
gb_default.fit(X_train, y_train)
t_fit = perf_counter() - t0

y_pred = gb_default.predict(X_test)
m = metrics_row(y_test, y_pred)

df_gb_default = pd.DataFrame([{
    'Modelo': 'Gradient Boosting (default)',
    'CV Accuracy': cv_gb.mean(),
    'Test Accuracy': m['Test Accuracy'],
    'F1 (w)': m['F1 (w)'],
    'Tiempo (s)': t_fit
}])
df_gb_default


,Modelo,CV Accuracy,Test Accuracy,F1 (w),Tiempo (s)
0,Gradient Boosting (default),0.95727,0.966825,0.967405,2.086784


In [59]:
save_table(df_gb_default, 'gb_default')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/gb_default.tex')

### 6.2 Efecto de `n_estimators` y `learning_rate`

Existe un **trade-off** clásico entre estos dos parámetros: un `learning_rate` bajo necesita más estimadores para converger, pero suele generalizar mejor (cada árbol contribuye poco, evitando saltos bruscos). Un `learning_rate` alto converge rápido pero puede sobreajustar.

Probamos varias combinaciones para explorar este equilibrio.

In [60]:
configs = [
    (50,  0.1), (100, 0.1), (200, 0.1),
    (50,  0.5), (100, 0.05), (200, 0.01)
]

rows = []
for n_est, lr in configs:
    gb = GradientBoostingClassifier(
        n_estimators=n_est, learning_rate=lr, random_state=RANDOM_STATE
    )
    cv = cross_val_score(gb, X_train, y_train, cv=5, scoring='accuracy')
    gb.fit(X_train, y_train)
    rows.append({
        'n_estimators': n_est,
        'learning_rate': lr,
        'CV Accuracy': cv.mean(),
        'Test Accuracy': gb.score(X_test, y_test)
    })

df_gb_nlr = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
df_gb_nlr


,n_estimators,learning_rate,CV Accuracy,Test Accuracy
2,200,0.10,0.959644,0.971564
3,50,0.50,0.957864,0.954976
1,100,0.10,0.957270,0.966825
0,50,0.10,0.945401,0.943128
4,100,0.05,0.943027,0.945498
5,200,0.01,0.906825,0.900474


In [61]:
save_table(df_gb_nlr, 'gb_n_lr')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/gb_n_lr.tex')

La combinación `(n_estimators=200, learning_rate=0.1)` lidera con un **CV = 95.96 % y Test = 97.16 %**. Reducir el learning_rate a 0.01 con solo 200 árboles resulta insuficiente (90.07 %): el modelo no ha convergido porque necesitaría muchos más estimadores. Un learning_rate alto (0.5) con pocos árboles (50) ofrece un buen compromiso rápido pero no alcanza el rendimiento máximo.

### 6.3 Efecto de `max_depth`

En Gradient Boosting, `max_depth` controla la complejidad de cada árbol individual. A diferencia de Random Forest (donde los árboles suelen ser profundos), en boosting se prefieren **árboles poco profundos** (3–5 niveles) porque el propio proceso iterativo se encarga de capturar interacciones complejas.

Árboles con `max_depth=1` (stumps) solo capturan efectos principales. Con `max_depth=3` se modelan interacciones de hasta 3 variables.

In [62]:
rows = []
for depth in [1, 2, 3, 5]:
    gb = GradientBoostingClassifier(
        max_depth=depth, n_estimators=100, learning_rate=0.1,
        random_state=RANDOM_STATE
    )
    cv = cross_val_score(gb, X_train, y_train, cv=5, scoring='accuracy')
    gb.fit(X_train, y_train)
    rows.append({
        'max_depth': depth,
        'Train Acc': gb.score(X_train, y_train),
        'CV Accuracy': cv.mean(),
        'Test Accuracy': gb.score(X_test, y_test),
    })

df_gb_depth = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
df_gb_depth


,max_depth,Train Acc,CV Accuracy,Test Accuracy
2,3,1.000000,0.957270,0.966825
3,5,1.000000,0.956083,0.959716
1,2,0.990504,0.938872,0.940758
0,1,0.878932,0.843917,0.841232


In [63]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(df_gb_depth['max_depth'], df_gb_depth['Train Acc'], marker='o', label='Train')
ax.plot(df_gb_depth['max_depth'], df_gb_depth['CV Accuracy'], marker='o', label='CV')
ax.plot(df_gb_depth['max_depth'], df_gb_depth['Test Accuracy'], marker='o', label='Test')
ax.set_xlabel('max_depth')
ax.set_ylabel('Accuracy')
ax.set_title('Gradient Boosting — Efecto de max_depth')
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(IMAGES_DIR / 'gb_depth_curve.png')


In [64]:
save_table(df_gb_depth, 'gb_depth')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/gb_depth.tex')

Con `max_depth=3` se obtiene el mejor CV (95.73 %) y test (96.68 %). Profundidades mayores (`max_depth=5`) alcanzan 100 % en train pero la CV y test empiezan a descender ligeramente, indicando **sobreajuste incipiente**. Los stumps (`max_depth=1`) solo alcanzan 84.4 %, confirmando que se necesitan interacciones entre variables para clasificar correctamente las 7 categorías.

### 6.4 Grid Search (Gradient Boosting)

Combinamos `n_estimators`, `learning_rate`, `max_depth` y `subsample` en una búsqueda exhaustiva. El parámetro `subsample < 1.0` introduce *stochastic gradient boosting*, donde cada árbol se entrena con una fracción aleatoria de los datos, lo que puede mejorar la generalización y reducir el sobreajuste.

In [65]:
param_grid_gb = {
    'n_estimators':  [100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.5],
    'max_depth':     [1, 2, 3],
    'subsample':     [0.8, 1.0]
}

grid_gb = GridSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_STATE),
    param_grid_gb,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    refit=True
)

t0 = perf_counter()
grid_gb.fit(X_train, y_train)
t_grid = perf_counter() - t0

print("Mejores parámetros:", grid_gb.best_params_)
print(f"Mejor CV score: {grid_gb.best_score_:.4f}")
print(f"Test score: {grid_gb.best_estimator_.score(X_test, y_test):.4f}")
print(f"Tiempo GridSearch: {t_grid:.3f}s")


Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Mejor CV score: 0.9596
Test score: 0.9716
Tiempo GridSearch: 44.456s


In [66]:
cvres_gb = pd.DataFrame(grid_gb.cv_results_).sort_values('mean_test_score', ascending=False).head(10)

rows = []
for _, r in cvres_gb.iterrows():
    params = {
        'n_estimators': int(r['param_n_estimators']),
        'learning_rate': float(r['param_learning_rate']),
        'max_depth': int(r['param_max_depth']),
        'subsample': float(r['param_subsample'])
    }
    model = GradientBoostingClassifier(random_state=RANDOM_STATE, **params)
    model.fit(X_train, y_train)
    rows.append({
        **params,
        'CV Acc': float(r['mean_test_score']),
        'Test Acc': model.score(X_test, y_test),
    })

df_gb_grid = pd.DataFrame(rows)
df_gb_grid


,n_estimators,learning_rate,max_depth,subsample,CV Acc,Test Acc
0,200,0.10,3,1.0,0.959644,0.971564
1,200,0.50,3,0.8,0.958457,0.957346
2,200,0.50,3,1.0,0.958457,0.962085
3,200,0.10,3,0.8,0.958457,0.962085
4,100,0.50,3,0.8,0.957864,0.969194
5,200,0.05,3,1.0,0.957864,0.959716
6,200,0.05,3,0.8,0.957270,0.945498
7,100,0.10,3,1.0,0.957270,0.966825
8,100,0.50,3,1.0,0.956677,0.962085
9,100,0.10,3,0.8,0.956083,0.952607


In [67]:
save_table(df_gb_grid, 'gb_grid')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/gb_grid.tex')

### 6.5 AdaBoost (stumps)

AdaBoost con *decision stumps* (`max_depth=1`) es un clasificador de boosting clásico. Cada stump solo puede usar **una feature y un umbral**, lo que lo convierte en un clasificador extremadamente débil. AdaBoost pondera las muestras según si fueron bien o mal clasificadas, forzando a los nuevos stumps a enfocarse en los ejemplos difíciles.

> **Nota**: Para un problema multiclase con 7 categorías, los stumps son demasiado simples como estimadores base. Esperamos resultados pobres comparados con Gradient Boosting, que usa árboles más profundos.

In [68]:
def make_adaboost(n_estimators: int, learning_rate: float) -> AdaBoostClassifier:
    """Compatibilidad sklearn: estimator (nuevo) vs base_estimator (antiguo)."""
    stump = DecisionTreeClassifier(max_depth=1, random_state=RANDOM_STATE)
    try:
        return AdaBoostClassifier(
            estimator=stump,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=RANDOM_STATE
        )
    except TypeError:
        return AdaBoostClassifier(
            base_estimator=stump,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=RANDOM_STATE
        )

ada_configs = [
    (50,  1.0),
    (100, 1.0),
    (100, 0.5),
    (200, 0.1)
]

rows = []
for n_est, lr in ada_configs:
    ada = make_adaboost(n_est, lr)
    cv = cross_val_score(ada, X_train, y_train, cv=5, scoring='accuracy')
    t0 = perf_counter()
    ada.fit(X_train, y_train)
    t_fit = perf_counter() - t0
    rows.append({
        'n_estimators': n_est,
        'learning_rate': lr,
        'CV Accuracy': cv.mean(),
        'Test Accuracy': ada.score(X_test, y_test),
        'Tiempo (s)': t_fit
    })

df_ada = pd.DataFrame(rows).sort_values('CV Accuracy', ascending=False)
df_ada


,n_estimators,learning_rate,CV Accuracy,Test Accuracy,Tiempo (s)
1,100,1.0,0.468843,0.407583,0.197057
2,100,0.5,0.467656,0.497630,0.194547
3,200,0.1,0.455786,0.433649,0.385678
0,50,1.0,0.439169,0.421801,0.096884


In [69]:
save_table(df_ada, 'adaboost')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/adaboost.tex')

### 6.6 Evaluación final (Parte 4)

Comparamos los modelos de boosting (GB default, GB mejor configuración y AdaBoost) junto con el mejor Random Forest como referencia. Incluimos la matriz de confusión del mejor modelo de esta sección.

In [70]:
best_gb = grid_gb.best_estimator_
best_gb.fit(X_train, y_train)
y_pred_gb = best_gb.predict(X_test)

best_ada_params = df_ada.iloc[0][['n_estimators','learning_rate']].to_dict()
best_ada = make_adaboost(int(best_ada_params['n_estimators']), float(best_ada_params['learning_rate']))
best_ada.fit(X_train, y_train)
y_pred_ada = best_ada.predict(X_test)

y_pred_rf = best_rf.predict(X_test)

df_gb_summary = pd.DataFrame([
    {'Modelo': 'Random Forest (mejor config)', **metrics_row(y_test, y_pred_rf)},
    {'Modelo': 'Gradient Boosting (default)', **metrics_row(y_test, gb_default.predict(X_test))},
    {'Modelo': 'Gradient Boosting (mejor config)', **metrics_row(y_test, y_pred_gb)},
    {'Modelo': 'AdaBoost (mejor config)', **metrics_row(y_test, y_pred_ada)},
]).sort_values('F1 (w)', ascending=False)

df_gb_summary


,Modelo,Test Accuracy,Precision (w),Recall (w),F1 (w)
2,Gradient Boosting (mejor config),0.971564,0.974512,0.971564,0.971800
1,Gradient Boosting (default),0.966825,0.971119,0.966825,0.967405
0,Random Forest (mejor config),0.954976,0.956814,0.954976,0.955283
3,AdaBoost (mejor config),0.407583,0.375229,0.407583,0.380158


In [71]:
save_table(df_gb_summary, 'gb_summary')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/gb_summary.tex')

In [72]:
best_part4_name = df_gb_summary.iloc[0]['Modelo']
if best_part4_name == 'Gradient Boosting (mejor config)':
    y_pred_best = y_pred_gb
elif best_part4_name == 'AdaBoost (mejor config)':
    y_pred_best = y_pred_ada
elif best_part4_name == 'Gradient Boosting (default)':
    y_pred_best = gb_default.predict(X_test)
else:
    y_pred_best = y_pred_rf

plot_confusion(
    y_test, y_pred_best, class_names,
    filename='confusion_matrix_gb.png',
    title=f'Matriz de Confusión — Mejor modelo Parte 4: {best_part4_name}'
)


**Análisis de resultados (Parte 4):**

**Gradient Boosting** domina claramente esta sección con un **97.16 % de Test Accuracy** (F1 = 0.9718), superando tanto al GB por defecto (96.68 %) como al mejor Random Forest (95.50 %). La estrategia de boosting — construir árboles secuenciales que corrigen los errores del anterior — resulta especialmente eficaz en este dataset.

Los parámetros óptimos son `learning_rate=0.1`, `max_depth=3` y `n_estimators=200`, confirmando que **árboles poco profundos combinados con muchas iteraciones** es la receta clásica de Gradient Boosting.

**AdaBoost con stumps** fracasa estrepitosamente (≈ 47 % CV, 40 % test), lo cual era previsible: un stump solo puede dividir el espacio con un único corte, totalmente insuficiente para discriminar entre 7 clases de obesidad. En un problema binario, AdaBoost con stumps puede funcionar razonablemente, pero el salto a 7 clases lo vuelve inviable con estimadores tan débiles.

---

## 7. Comparación global de mejores modelos

Finalmente, seleccionamos el **mejor modelo de cada parte** y los enfrentamos en una comparación directa. Para cada modelo calculamos CV Accuracy (5-fold), Test Accuracy, Precision, Recall, F1 ponderado y tiempo de entrenamiento. Esto nos permite evaluar no solo la calidad predictiva sino también el coste computacional de cada enfoque.

Los modelos seleccionados son:
- **SVM (best RBF)**: Pipeline con StandardScaler + SVC(kernel='rbf', C=100, gamma=0.01)
- **Decision Tree (best pre-pruning)**: mejor configuración por GridSearchCV
- **Random Forest (best)**: 200 árboles, todas las features, sin límite de profundidad
- **Gradient Boosting (best)**: lr=0.1, max_depth=3, n_estimators=200, subsample=1.0
- **AdaBoost (best)**: incluido para contrastar su bajo rendimiento con stumps

In [73]:
# Seleccionamos mejores modelos por bloque
best_models = {
    'SVM (best RBF)': grid_rbf.best_estimator_,
    'Decision Tree (best pre-pruning)': grid_tree.best_estimator_,
    'Random Forest (best)': grid_rf.best_estimator_,
    'Gradient Boosting (best)': grid_gb.best_estimator_,
    'AdaBoost (best)': best_ada,
}

rows = []
for name, model in best_models.items():
    # CV accuracy
    cv = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean, cv_std = cv.mean(), cv.std()

    # Fit time
    t0 = perf_counter()
    model.fit(X_train, y_train)
    fit_time = perf_counter() - t0

    y_pred = model.predict(X_test)
    m = metrics_row(y_test, y_pred)

    rows.append({
        'Modelo': name,
        'CV Accuracy': cv_mean,
        'CV Std': cv_std,
        **m,
        'Tiempo (s)': fit_time
    })

df_final = pd.DataFrame(rows).sort_values('F1 (w)', ascending=False)
df_final


,Modelo,CV Accuracy,CV Std,Test Accuracy,Precision (w),Recall (w),F1 (w),Tiempo (s)
3,Gradient Boosting (best),0.959644,0.013083,0.971564,0.974512,0.971564,0.971800,4.069379
2,Random Forest (best),0.951929,0.016828,0.954976,0.956814,0.954976,0.955283,0.239856
0,SVM (best RBF),0.940059,0.018234,0.943128,0.943953,0.943128,0.943202,0.040027
1,Decision Tree (best pre-pruning),0.928783,0.012589,0.921801,0.925161,0.921801,0.922354,0.006653
4,AdaBoost (best),0.468843,0.027836,0.407583,0.375229,0.407583,0.380158,0.193457


In [74]:
save_table(df_final[['Modelo','CV Accuracy','Test Accuracy','Precision (w)','Recall (w)','F1 (w)','Tiempo (s)']], 'final_comparison')

WindowsPath('C:/Users/jordi/Documents/UNI/IA/3erAnyo/2doCuatri/aprendizaje_avanzado/practicas/practica_1/tables/final_comparison.tex')

In [75]:
best_name = df_final.iloc[0]['Modelo']
best_model = best_models[best_name]
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)

print('Mejor modelo global:', best_name)
print(classification_report(y_test, y_pred_best, target_names=class_names))

plot_confusion(
    y_test, y_pred_best, class_names,
    filename='confusion_matrix_final.png',
    title=f'Matriz de Confusión — Mejor modelo global: {best_name}'
)


Mejor modelo global: Gradient Boosting (best)
                     precision    recall  f1-score   support

Insufficient_Weight       1.00      0.91      0.95        54
      Normal_Weight       0.86      1.00      0.93        57
     Obesity_Type_I       0.99      0.99      0.99        70
    Obesity_Type_II       0.98      0.98      0.98        60
   Obesity_Type_III       1.00      1.00      1.00        65
 Overweight_Level_I       1.00      0.91      0.95        58
Overweight_Level_II       0.98      1.00      0.99        58

           accuracy                           0.97       422
          macro avg       0.97      0.97      0.97       422
       weighted avg       0.97      0.97      0.97       422



**Análisis global y conclusiones:**

El ranking final confirma que **Gradient Boosting** es el mejor modelo para este dataset, con un **97.16 % de Test Accuracy** y un F1 ponderado de 0.9718. Le sigue **Random Forest** con 95.50 %, luego **SVM (RBF)** con 94.31 %, **Decision Tree** con 92.18 % y finalmente **AdaBoost** con un rendimiento muy pobre (40.76 %).

Observaciones clave:

1. **La complejidad del ensemble importa**: de un árbol individual (92 %) a un bosque (95 %) y a boosting (97 %), cada nivel de sofisticación mejora el rendimiento.
2. **SVM es competitivo** con un 94 % sin ser ensemble, pero requiere un escalado cuidadoso y es más lento de optimizar.
3. **AdaBoost con stumps no es viable** para problemas multiclase complejos. Sería necesario usar estimadores base más profundos para que fuera competitivo.
4. **El classification report** muestra que Obesity_Type_III alcanza F1 = 1.00 (perfectamente separable), mientras que las categorías con mayor confusión son Normal_Weight e Insufficient_Weight, que comparten rangos de peso similares.
5. **Trade-off tiempo/rendimiento**: SVM es el más rápido de los buenos modelos (0.04s), mientras que GB necesita 4s. Para este dataset la diferencia es negligible, pero en producción podría ser relevante.